<a href="https://colab.research.google.com/github/Recollectionss/University/blob/main/4-course/rms/lab-2/rms_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# ============================================
#        ASTRO-SONIFICATION — Google Colab + WAV
# ============================================

# install required libraries
!pip install mido python-rtmidi pyFluidSynth --quiet

import pandas as pd
import math
from mido import Message, MidiFile, MidiTrack
from google.colab import files

# --------------------------------------------
#   1. Cosmic Instrument Set (General MIDI)
# --------------------------------------------
INSTRUMENTS = {
    80:  "Square Lead Synth",
    89:  "Warm Pad",
    91:  "Space Voice Pad",
    94:  "Halo Pad",
    95:  "Sweep Pad",
    102: "FX Atmosphere",
    103: "FX Brightness",
    104: "Ethereal Sitar",
    118: "Cosmic Synth Drum"
}

# --------------------------------------------
#   2. Helper: map range → range
# --------------------------------------------
def map_value(value, min_value, max_value, min_result, max_result):
    if value is None or (isinstance(value, float) and math.isnan(value)):
        return min_result
    if max_value == min_value:
        return min_result
    return min_result + (value - min_value) / (max_value - min_value) * (max_result - min_result)

# --------------------------------------------
#   3. Load CSV
# --------------------------------------------
print("Upload a CSV file with columns:")
print("name, radius, mass, temperature, orbit_period, rotation_period\n")
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)

required_columns = {'name', 'radius', 'mass', 'temperature', 'orbit_period', 'rotation_period'}
missing = required_columns - set(df.columns)
if missing:
    raise ValueError(f"❌ Missing required columns: {missing}")

df = df.sort_values(by=['orbit_period'], ascending=False)
print(f"\nLoaded {len(df)} astronomical objects:")
display(df.head())

# --------------------------------------------
#   4. Choose instrument
# --------------------------------------------
def select_instrument():
    print("\n🎵 Select an instrument:\n")
    for prog, name in INSTRUMENTS.items():
        print(f"{prog:>3} — {name}")
    while True:
        try:
            choice = int(input("\nEnter instrument number: "))
            if choice in INSTRUMENTS:
                print(f"Selected: {INSTRUMENTS[choice]}")
                return choice
            print("Invalid number, try again.")
        except:
            print("Enter a valid integer number.")

instrument_choice = select_instrument()

# --------------------------------------------
#   5. Normalize data ranges
# --------------------------------------------
min_temp, max_temp = df['temperature'].min(), df['temperature'].max()
min_radius, max_radius = df['radius'].min(), df['radius'].max()
min_mass, max_mass = df['mass'].min(), df['mass'].max()
min_orbit, max_orbit = df['orbit_period'].min(), df['orbit_period'].max()
min_rot, max_rot = df['rotation_period'].min(), df['rotation_period'].max()

# --------------------------------------------
#   6. Generate MIDI
# --------------------------------------------
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)
track.append(Message('program_change', program=instrument_choice, time=0))

print("\nGenerating MIDI...\n")
for _, row in df.iterrows():
    note = int(map_value(row['temperature'], min_temp, max_temp, 40, 100))
    velocity = int(map_value(row['radius'], min_radius, max_radius, 50, 127))
    duration = int(map_value(row['orbit_period'], min_orbit, max_orbit, 240, 1800))
    delay = int(map_value(row['rotation_period'], min_rot, max_rot, 60, 600))
    track.append(Message('note_on', note=note, velocity=velocity, time=delay))
    track.append(Message('note_off', note=note, velocity=velocity, time=duration))
    print(f"{row['name']}: note={note}, vel={velocity}, dur={duration}, delay={delay}")

midi_name = f"astro_sonification_{INSTRUMENTS[instrument_choice].replace(' ', '_')}.mid"
mid.save(midi_name)
print(f"\n✅ MIDI file generated: {midi_name}")

# --------------------------------------------
#   7. Convert MIDI to WAV
# --------------------------------------------
!apt-get install -y fluidsynth timidity >/dev/null 2>&1
wav_name = midi_name.replace(".mid", ".wav")

import subprocess
# Use FluidSynth with default soundfont
soundfont = "/usr/share/sounds/sf2/FluidR3_GM.sf2"
subprocess.run(f"fluidsynth -ni {soundfont} {midi_name} -F {wav_name} -r 44100", shell=True, check=True)
print(f"✅ WAV file generated: {wav_name}")

# --------------------------------------------
#   8. Download WAV
# --------------------------------------------
files.download(wav_name)


Upload a CSV file with columns:
name, radius, mass, temperature, orbit_period, rotation_period



Saving saturn_system.csv to saturn_system.csv

Loaded 10 astronomical objects:


,name,radius,mass,temperature,orbit_period,rotation_period
0,Saturn,60268,568.00000,5778,29.4571,0.444
9,Phoebe,106,0.00008,75,0.3760,0.374
7,Iapetus,734,0.00100,93,0.0793,79.321
1,Titan,2575,1.34500,94,0.0377,15.945
2,Rhea,763,0.02300,72,0.0230,4.518



🎵 Select an instrument:

 80 — Square Lead Synth
 89 — Warm Pad
 91 — Space Voice Pad
 94 — Halo Pad
 95 — Sweep Pad
102 — FX Atmosphere
103 — FX Brightness
104 — Ethereal Sitar
118 — Cosmic Synth Drum

Enter instrument number: 94
Selected: Halo Pad

Generating MIDI...

Saturn: note=100, vel=127, dur=1800, delay=60
Phoebe: note=40, vel=50, dur=259, delay=60
Iapetus: note=40, vel=50, dur=244, delay=600
Titan: note=40, vel=53, dur=241, delay=166
Rhea: note=40, vel=50, dur=241, delay=88
Hyperion: note=40, vel=50, dur=241, delay=61
Tethys: note=40, vel=50, dur=240, delay=70
Dione: note=40, vel=50, dur=240, delay=76
Enceladus: note=40, vel=50, dur=240, delay=66
Mimas: note=40, vel=50, dur=240, delay=63

✅ MIDI file generated: astro_sonification_Halo_Pad.mid
✅ WAV file generated: astro_sonification_Halo_Pad.wav


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>